# Functional Test 3.1.2 - NVMEs

This Jupyter notebook will allow you to create VMs on different sites and worker nodes consistent with requirements for test 3.1.2 for testing NVMe attachment.

## Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../fablib_api/configure_environment/configure_environment.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

**This only needs to be done once.**

## Step 2: Import the FABlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

## Step 3 Check your existing slices

Since testing can get confusing, check what slices you actually have. It may print nothing if you have no active slices.

In [ ]:
try:
    for slice in fablib.get_slices():
        print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

## Step 4: Create the test Slice

This creates a VM with two NVMes on a specific worker at a specific site (there are typically at least 2 available at each worker). If you are unsure, the generated ads for each site ([in JSON format](https://github.com/fabric-testbed/aggregate-ads/tree/main/JSON)) can help. 

**The code to create the slice will auto-refresh until the slice is created or it fails**

In [ ]:
from datetime import datetime
from dateutil import tz

name='Node1'
nvme_name1='nvme1'
nvme_name2='nvme2'
site='TACC'
# since all workers have a standard naming scheme, you can just change the worker
# to move from worker to worker
worker=f'{site.lower()}-w1.fabric-testbed.net'
cores=10
ram=20
disk=50
slice_name=f"Slice Test 3.1.2-NVMe {site} on {worker} on {datetime.now()}"

In [ ]:
try:
    #Create Slice
    print(f'Creating slice {slice_name}')
    slice = fablib.new_slice(name=slice_name)

    # Add node
    node = slice.add_node(name=name, site=site, host=worker, cores=cores, ram=ram, disk=disk)
    
    #Add an NVMe Drive
    node.add_component(model='NVME_P4510', name=nvme_name1)
    #Add another NVMe Drive
    node.add_component(model='NVME_P4510', name=nvme_name2)

    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

## Step 5: Observe the Slice's Attributes

### Print the slice 

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

### Print the node

Each node in the slice has a set of get functions that return the node's attributes. Use the returned `SSH Command` string to check the node. You can do it from a Bash launched inside the Jupyter container.


In [ ]:
try:
    node = slice.get_node(name) 
    print(f"{node}")
  
    nvme1 = node.get_component(nvme_name1)
    print(f"{nvme1}")
    
    nvme2 = node.get_component(nvme_name2)
    print(f"{nvme2}")
except Exception as e:
    print(f"Exception: {e}")

### NVMe PCI Devices

Run the command <code>lspci</code> to see your NVMe PCI device(s). 

View node1's NVMe's

In [ ]:
command = "sudo dnf install -q -y pciutils && lspci"
try:
    stdout, stderr = node.execute(command)
    print(f"stdout: {stdout}")
except Exception as e:
    print(f"Exception: {e}")

## Step 6: Configure NVMe drives

NVMe storage is provided as bare PCI block devices and will need to be partitioned, formated, and mounted before use. Observe the devices.

In [ ]:
try:
    nvme1.configure_nvme()
    nvme2.configure_nvme()
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
command = "sudo fdisk -l"
try:
    stdout, stderr = node.execute(command)
    print(f"stdout: {stdout}")
except Exception as e:
    print(f"Exception: {e}")

## Step 7: Cleanup Your Slice

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")